In [238]:
import requests 
import plotly.express as px
import plotly.graph_objects as pgo
import plotly.offline as pyo
import pandas as pd

In [239]:
all_scales = ['goitre39', 'hyperthyroidscore', 'hypothyroidscore', 'eyescore', 'tirednessscore',
                    'cognitivescore', 'anxietyscore', 'depressivityscore', 'emotionalscore', 'sociallifescore',
                    'dailylifescore','cosmeticscore','overallqolscore','compositescore']  

###fetch_n_recordIDs_for_category

In [240]:
def fetch_n_recordIDs_for_category(scale, recordIDs): # returnerer en liste af scores for en liste af recordIDs
  api_call = { 
        'token': 'DD0E88BF0EF8E12ACD577F664A515337', # api token
        'content': 'record',
        'format': 'json',
        'type': 'flat',
        'records[0]': '', # initally tom
        'fields[0]': f'{scale}', # hvilken scale du vil have svar ud for
        'forms[0]': 'record_id', # specifies we are searching with a recordID
        'returnFormat': 'json'}
    
  scale_values = [] 

  for recID in recordIDs:
    api_call['records[0]'] = recID
    response = requests.post('https://redcap.regionh.dk/api/', data=api_call) 
    if (str(response.status_code) == '200'): # if the HTML requests is succesfull
      json_response = response.json()
      value = json_response[0][scale]
      
      scale_values.append(value)
  df = pd.DataFrame({"RecordID": recordIDs, f'{scale}' : scale_values})
  #df.set_index('RecordID', inplace=True) # gør noget mærkeligt jeg ikke forstår
  
  return df
    

###fetch_n_categories_for_recordID

In [241]:
def fetch_n_categories_for_recordID(recordID, scales): # contacts the API and returns a pandas dataframe list with thypro scores for a recordID
    api_call = { 
        'token': 'DD0E88BF0EF8E12ACD577F664A515337', # api token
        'content': 'record',
        'format': 'json',
        'type': 'flat',
        'records[0]': f'{recordID}', # hvilken record du vil svar ud for
        'fields[0]': '', # er initially None
        'forms[0]': 'record_id', # specifies we are searching with a recordID
        'returnFormat': 'json'}
        
    thypro_scale_values= {}
    for scale in scales: # makes the list for holding the data
        thypro_scale_values[f'{scale}'] = 0 # tilføjer skalaen til dicten

        api_call['fields[0]'] = scale
        response = requests.post('https://redcap.regionh.dk/api/', data=api_call) # contacts the API and gets the response

        if (str(response.status_code) == '200'): # if the API call is succesfull
            json_response = response.json()

            scale_value = json_response[0][scale] 
            if(scale_value == '-1' or scale_value == ''): # if a value is not present, it returns -1 or maybe ''?? 
                scale_value = 0
        
            thypro_scale_values[scale] = scale_value # updates the result from the API call in the stored dict
        else:
            print(f"HTTP get request failed with {str(response.status_code)} status code")

    values_raw = [float(x) for x in list(thypro_scale_values.values())] # a list with only the values from thypro_scale_values dict
    scales_raw = [x for x in list(thypro_scale_values.keys())]
     
    return pd.DataFrame({"Kategorier": scales_raw, "Score":values_raw}) 

#Plot funktioner

In [242]:
 from google.colab import files

In [243]:
def plot_line(recordID_list, categories): # plots a line chart. Takes a list of records, and what categories to display
  trace_dict = {}
  for cat in categories:
    value_for_category = fetch_n_recordIDs_for_category(cat, recordID_list) # pandas dt
    trace = pgo.Scatter(x = value_for_category.iloc[:,0], 
                        y = value_for_category.iloc[:,1],
                        mode = 'lines',
                        name =f'{cat}')
    trace_dict[f"{cat}%s" %cat] = trace

  trace_list=list(trace_dict.values()) # laver en liste med de forskellige traces 

  layout = pgo.Layout(title = "ThyPRO Scores")
  figure = pgo.Figure(data = trace_list, layout = layout)
  pyo.plot(figure)

  files.download('temp-plot.html') 

In [263]:
def plot_radar(record_IDs, categories): # function for plotting ThyPRO data in radar chart
    fig = pgo.Figure() 
    for rec in record_IDs:
      df = fetch_n_categories_for_recordID(rec, categories) 
      fig.add_trace(pgo.Scatterpolar(r=df.iloc[:,1], theta=categories, name=f'{rec}'))
    
    fig.update_layout(polar=dict(radialaxis=dict(visible=True,range=[0, 100])),showlegend=True)
    fig.show()

# Main

In [246]:
plot_line([26,27,28], all_scales)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [264]:
plot_radar([25,26,27], all_scales) # plots the radar chart